<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# ✂️ PySpark - split() - פיצול מחרוזות למערכים
[PySpark Convert String to Array Column](https://sparkbyexamples.com/pyspark/pyspark-convert-string-to-array-column/)
## תוכן עניינים
1. [הקדמה](#introduction)
2. [תחביר split()](#syntax)
3. [המרת עמודת מחרוזת למערך](#convert-string-to-array)
4. [שימוש ב-split() עם withColumn()](#with-column)
5. [שימוש ב-split() עם SQL Query](#with-sql)
6. [דוגמה מלאה](#complete-example)
7. [מקורות](#references)

## 🎯 הקדמה <a id='introduction'></a>

**PySpark SQL** מספקת פונקציה `split()` מתוך מודול `pyspark.sql.functions`. פונקציה זו משמשת להמרת עמודת מחרוזת ל-עמודת מערך.

הפונקציה מפצלת מחרוזת על בסיס תווי הפרדה (delimiter) כמו פסיק, רווח, צינור וכו', ומחזירה אובייקט מסוג `pyspark.sql.Column` מסוג Array.

## 📝 תחביר split() <a id='syntax'></a>

```python
pyspark.sql.functions.split(str, pattern, limit=-1)
```

### פרמטרים:

- **str**: שם העמודה המכילה את המחרוזת שממנה אתה רוצה לחלץ תת-מחרוזת
- **pattern**: תבנית הפרדה (delimiter). אם לא מצוינת, הפונקציה תחלץ מעמדת הפיצול עד סוף המחרוזת
- **limit** (אופציונלי): מספר האלמנטים המקסימלי להחזרה. אם לא מצוין, הפונקציה תחזיר את כל האלמנטים

## 📦 ייבוא ספריות והכנת נתונים

ראשית, נייבא את הספריות הנדרשות וניצור DataFrame לדוגמה עם עמודת שמות מופרדת בפסיקים:

In [ ]:
from pyspark.sql import SparkSession

# יצירת Spark Session
spark = SparkSession.builder.appName("stringoperations").getOrCreate()

# יצירת DataFrame עם נתוני שמות
data = [
    ("James, A, Smith", "2018", "M", 3000),
    ("Michael, Rose, Jones", "2010", "M", 4000),
    ("Robert, K, Williams", "2010", "M", 4000),
    ("Maria, Anne, Jones", "2005", "F", 4000),
    ("Jen, Mary, Brown", "2010", "", -1)
]

columns = ["name", "dob_year", "gender", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.printSchema()
df.show(truncate=False)

## 🔄 המרת עמודת מחרוזת למערך <a id='convert-string-to-array'></a>

נייבא את `split` ו-`col` ונשתמש בפונקציה `split()` על עמודת ה-name כדי לפצל אותה למערך על בסיס פסיקים:

In [ ]:
from pyspark.sql.functions import split, col

# פיצול עמודת 'name' למערך
df2 = df.select(
    split(col("name"), ",").alias("NameArray")
).alias("NameArray")

df2.printSchema()
df2.show()

### 📋 הסבר הפלט:

כפי שאתה רואה בפלט למעלה, עמודת NameArray היא מסוג array המכילה אלמנטים מסוג string.

## 🔨 שימוש ב-split() עם withColumn() <a id='with-column'></a>

אפשר להשתמש בפונקציה `split()` בתוך מתודת `withColumn()` כדי ליצור עמודה חדשה עם מערך על ה-DataFrame.

אם אינך זקוק לעמודה המקורית, השתמש ב-`drop()` כדי להסיר אותה:

In [ ]:
# פיצול עמודת 'name' לשלושה חלקים: שם פרטי, שם אמצע, שם משפחה
df = df.withColumn("name_array", split(df["name"], ","))

# הצגת DataFrame המעודכן
df.show(truncate=False)

## 📊 שימוש ב-split() עם SQL Query <a id='with-sql'></a>

לחלופין, אתה יכול לכתוב את אותו הדוגמה באמצעות שאילתת SQL.

ראשית, צור Temporary View באמצעות `createOrReplaceTempView()` ו-`spark.sql()` כדי להריץ את שאילתת ה-SQL:

In [ ]:
# יצירת Temporary View
df.createOrReplaceTempView("PERSON")

# הרצת SQL Query
spark.sql("select SPLIT(name, ',') as NameArray from PERSON").show()

## 🎓 דוגמה מלאה <a id='complete-example'></a>

הנה דוגמה מקיפה המשלבת את כל מה שלמדנו:

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

data = [
    ("James, A, Smith", "2018", "M", 3000),
    ("Michael, Rose, Jones", "2010", "M", 4000),
    ("Robert, K, Williams", "2010", "M", 4000),
    ("Maria, Anne, Jones", "2005", "F", 4000),
    ("Jen, Mary, Brown", "2010", "", -1)
]

columns = ["name", "dob_year", "gender", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.printSchema()
df.show(truncate=False)

# שימוש ב-split() עם select
from pyspark.sql.functions import split, col
df2 = df.select(split(col("name"), ",").alias("NameArray")).alias("NameArray")
df2.printSchema()
df2.show()

# שימוש ב-split() עם withColumn
df2 = df.select(split(col("name"), ",").alias("NameArray"))
df2.printSchema()
df2.show()

# שימוש עם SQL
df.createOrReplaceTempView("PERSON")
spark.sql("select SPLIT(name, ',') as NameArray from PERSON").show()

## ✨ סיכום

בשיעור זה למדנו דרכים שונות לפיצול עמודת מחרוזת ב-PySpark DataFrame:

**נקודות מפתח:**
- ✅ `split()` - פיצול מחרוזת למערך על בסיס delimiter
- ✅ שימוש עם `select()` ליצירת DataFrame חדש
- ✅ שימוש עם `withColumn()` להוספת עמודה למערך קיים
- ✅ שימוש עם SQL query ו-`createOrReplaceTempView()`

**מתי להשתמש:**
- כאשר יש לך מחרוזות מופרדות בפסיקים, רווחים או תווי הפרדה אחרים
- כאשר אתה רוצה להמיר עמודה למערך לעיבוד נוסף
- כאשר אתה צריך לפצל שמות מלאים לחלקיהם

הגישה הזו שימושית במיוחד לטרנספורמציה של מחרוזות מופרדות במבנה מערך למטרות עיבוד נוספות!

## 📚 מקורות <a id='references'></a>

- [PySpark SQL Functions Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html)
- [SparkByExamples - PySpark split() Column into Multiple Columns](https://sparkbyexamples.com/pyspark/pyspark-convert-string-to-array-column/)